In [1]:
import torch.nn as nn
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=3, padding='same')
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.leaky_relu1 = nn.LeakyReLU(0.02)
        self.drop1 = nn.Dropout(dropout_prob)

        self.conv2 = nn.Conv1d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=3, padding='same')
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.leaky_relu2 = nn.LeakyReLU(0.02)
        self.drop2 = nn.Dropout(dropout_prob)

        self.pool = nn.AdaptiveAvgPool1d(1) 

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.leaky_relu1(out)
        out = self.drop1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.leaky_relu2(out)
        out = self.drop2(out)
        
        out = self.pool(out)
        out = out.view(out.size(0), -1) 
        out = self.fc(out)

        return out

In [ ]:
def training_model(model, optimizer, criterion, train_loader, test_loader, epochs, device):
    train_losses = []
    test_losses = []
    accuracy_values = []

    num_epochs = epochs
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_total = 0
        correct = 0
        test_total = 0
        test_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            train_total += labels.size(0)
        
        model.eval()
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                test_loss += loss.item() * inputs.size(0)
                predicted = (torch.sigmoid(outputs) > 0.5).float()

                correct += (predicted == labels).sum().item()
                test_total += labels.numel()

        train_loss = running_loss / train_total
        test_loss = test_loss / test_total
        accuracy = correct / test_total
        
        # Store losses for plotting
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        accuracy_values.append(accuracy*100)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy*100:.4f}%')
    
    return model

In [ ]:
def test_model(model, test_loader, device):
    y_true = []
    y_pred = []
    correct = 0
    test_total = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (torch.sigmoid(outputs) > 0.5).float()

            correct += (predicted == labels).sum().item()
            test_total += labels.numel()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
        
    accuracy = correct / test_total
    print(f'Accuracy: {accuracy}')
    precision = precision_score(y_true, y_pred, average='samples')
    recall = recall_score(y_true, y_pred, average='samples')
    f1 = f1_score(y_true, y_pred, average='samples')
    return accuracy, precision, recall, f1